# Setting up MetaData with Table object

In [1]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

# Declaring the table

In [2]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

# 'Table' -- represents a DB table and assigns itself to a Metadata collection
# 'Column' -- represents a column in a database table, and assigns itself to a Table object.
# 'Integer', 'String' -- these classes represent SQL datatypes and can be passed to a Column, 
#             with or without necessarily being instantiated.

In [3]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [4]:
user_table.c.keys()

['id', 'name', 'fullname']

In [5]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [6]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

# Emitting DDL to the DB (~CREATE TABLE)

In [7]:
# create an engine object
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [8]:
metadata_obj.create_all(engine)

2024-10-03 14:04:05,215 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:05,216 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-03 14:04:05,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,217 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-10-03 14:04:05,218 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,218 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-03 14:04:05,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,219 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-10-03 14:04:05,221 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,222 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-10-03 14:04:05,223 INFO sqlalchemy.engine.Engine [no key 0.00089s] ()
2024-10-03 14:04:05,224 INFO sqlalchemy.engine.Engine 
C

# Establishing a Declarative Base

In [9]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [10]:
Base.metadata

MetaData()

In [11]:
Base.registry

# Declaring Mapped Classes

In [12]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

# Emitting DDL to the database from an ORM mapping

In [13]:
Base.metadata.create_all(engine)

2024-10-03 14:04:05,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:05,425 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-03 14:04:05,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,426 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-03 14:04:05,426 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,427 INFO sqlalchemy.engine.Engine COMMIT


# Table Reflection

In [14]:
# prepare the table named 'some_table' from previous section

from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()
    
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2024-10-03 14:04:05,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:05,583 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-10-03 14:04:05,584 INFO sqlalchemy.engine.Engine [generated in 0.00230s] ()
2024-10-03 14:04:05,585 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-10-03 14:04:05,586 INFO sqlalchemy.engine.Engine [generated in 0.00108s] [(1, 1), (2, 4)]
2024-10-03 14:04:05,587 INFO sqlalchemy.engine.Engine COMMIT
2024-10-03 14:04:05,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:05,589 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-10-03 14:04:05,591 INFO sqlalchemy.engine.Engine [cached since 0.005111s ago] [(6, 8), (9, 10)]
2024-10-03 14:04:05,591 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2024-10-03 14:04:05,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:05,717 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2024-10-03 14:04:05,718 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,720 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-10-03 14:04:05,720 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2024-10-03 14:04:05,721 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2024-10-03 14:04:05,722 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,723 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2024-10-03 14:04:05,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-03 14:04:05,724 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [16]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

# The insert() SQL Expression Construct

In [17]:
from sqlalchemy import insert

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [18]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [19]:
compiled = stmt.compile()

In [20]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [21]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2024-10-03 14:04:06,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:06,336 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-03 14:04:06,337 INFO sqlalchemy.engine.Engine [generated in 0.00184s] ('spongebob', 'Spongebob Squarepants')
2024-10-03 14:04:06,339 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
result.inserted_primary_key

(1,)

# INSERT usually generate the "values" clause automatically

In [23]:
# Simple insert() statement provide input for all columns in the table
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [24]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2024-10-03 14:04:06,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:06,503 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-03 14:04:06,504 INFO sqlalchemy.engine.Engine [generated in 0.00188s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2024-10-03 14:04:06,504 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2024-10-03 14:04:06,546 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:06,547 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-10-03 14:04:06,548 INFO sqlalchemy.engine.Engine [generated in 0.00210s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2024-10-03 14:04:06,549 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
# A true 'empty' INSERT that inserts only the 'defaults' for a table:
print(insert(user_table).values().compile(engine))

INSERT INTO user_account DEFAULT VALUES


# INSERT...RETURNING

In [27]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [28]:
# It can also be combined with Insert.from_select(), 
# as in the example below that builds upon the example stated in INSERT…FROM SELECT

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


# INSERT...FROM SELECT

In [29]:
# This construct is used when one wants to copy data from some other part of the database directly 
# into a new set of rows, without actually fetching and re-sending the data from the client

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


# *SELECT Statement examples*

In [30]:
# SELECT FROM Table

from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [31]:
# executing the SELECT statement

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2024-10-03 14:04:06,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:06,908 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-10-03 14:04:06,909 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2024-10-03 14:04:06,910 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
# ORM class

from sqlalchemy.orm import Session

stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2024-10-03 14:04:06,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:06,963 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-10-03 14:04:06,964 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2024-10-03 14:04:06,965 INFO sqlalchemy.engine.Engine ROLLBACK


# Setting the COLUMNS and FROM clause

In [33]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [34]:
# SELECT from individual columns
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [35]:
# SELECT from individual columns in a bit different way
print(select(user_table.c["name", "fullname"]))

SELECT user_account.name, user_account.fullname 
FROM user_account


# Selecting ORM Entities and Columns

In [36]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [37]:
row = session.execute(select(User)).first()

2024-10-03 14:04:07,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:04:07,324 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-10-03 14:04:07,325 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()


In [38]:
row

(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

In [39]:
# using method .scalars() which returns object
user = session.scalars(select(User)).first()

2024-10-03 14:04:07,420 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-10-03 14:04:07,421 INFO sqlalchemy.engine.Engine [cached since 0.09564s ago] ()


In [40]:
user

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [41]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [42]:
row = session.execute(select(User.name, User.fullname)).first()

2024-10-03 14:04:07,552 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2024-10-03 14:04:07,553 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()


In [43]:
row

('spongebob', 'Spongebob Squarepants')

In [44]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id).order_by(Address.id)
).all()

2024-10-03 14:04:07,649 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2024-10-03 14:04:07,650 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('sandy', Address(id=3, email_address='sandy@squirrelpower.org'))]

# Selecting from Labeled SQL Expression

In [45]:
from sqlalchemy import func, cast

stmt = select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")
        
# Also, Ordering or Grouping by a Label is also possible

2024-10-03 14:06:14,357 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:06:14,357 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2024-10-03 14:06:14,358 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2024-10-03 14:06:14,360 INFO sqlalchemy.engine.Engine ROLLBACK


# Selecting with Textual Column Expression

In [46]:
from sqlalchemy import text

stmt = select(text("'some phrase'"), user_table.c.name).order_by(user_table.c.name)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2024-10-03 14:19:04,596 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:19:04,597 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2024-10-03 14:19:04,597 INFO sqlalchemy.engine.Engine [generated in 0.00166s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2024-10-03 14:19:04,598 INFO sqlalchemy.engine.Engine ROLLBACK


In [47]:
# giving the label to literal_column
from sqlalchemy import literal_column

stmt = select(literal_column("'some phrase'").label("p"), user_table.c.name).order_by(
    user_table.c.name
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2024-10-03 14:24:21,694 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 14:24:21,694 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2024-10-03 14:24:21,696 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2024-10-03 14:24:21,697 INFO sqlalchemy.engine.Engine ROLLBACK


In [48]:
print(user_table.c.name == "squidward")

user_account.name = :name_1


# The WHERE clause

In [49]:
print(address_table.c.user_id > 10)

address.user_id > :user_id_1


In [50]:
print(select(user_table).where(user_table.c.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [52]:
# Multiple expressions with WHERE via AND v1

print(
    select(address_table.c.email_address)
    .where(user_table.c.name == "squidward")
    .where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [53]:
# Multiple expressions with WHERE via AND v2

print(
    select(address_table.c.email_address).where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id,
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [54]:
# Combining AND and OR in WHERE clause

from sqlalchemy import and_, or_

print(
    select(Address.email_address).where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id,
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


In [55]:
# Comparisons for single 'equality' comparisons against a single entity,
# also called FILTER BY.

print(select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


# Explicit FROM clauses and JOINs

In [56]:
# selecting from a single table
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


In [57]:
# selecting from multiple tables
# this do NOT work properly!!!
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


In [58]:
# JOINing tables in the proper way

# The first is the Select.join_from() method, which allows us to 
# indicate the left and right side of the JOIN explicitly

print(
    select(user_table.c.name, address_table.c.email_address).join_from(
        user_table, address_table
    )
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [59]:
# The other is the Select.join() method, which indicates only
# the right side of the JOIN, the left hand-side is inferred:

print(select(user_table.c.name, address_table.c.email_address).join(address_table))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [60]:
print(select(address_table.c.email_address).select_from(user_table).join(address_table))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


# Setting the ON Clause

In [61]:
print(
    select(address_table.c.email_address)
    .select_from(user_table)
    .join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


# OUTER and FULL join

In [62]:
print(select(user_table).join(address_table, isouter=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [63]:
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


# ORDER BY, GROUP BY, HAVING

In [64]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


In [65]:
# Ascending / descending is available from the ColumnElement.asc()
# and ColumnElement.desk()
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


In [66]:
from sqlalchemy import func

count_fn = func.count(user_table.c.id)
print(count_fn)

count(user_account.id)


In [67]:
# GROUP BY and HAVING

with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count"))
        .join(Address)
        .group_by(User.name)
        .having(func.count(Address.id) > 1)
    )
    print(result.all())

2024-10-03 15:27:33,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-03 15:27:33,532 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2024-10-03 15:27:33,533 INFO sqlalchemy.engine.Engine [generated in 0.00373s] (1,)
[('sandy', 2)]
2024-10-03 15:27:33,536 INFO sqlalchemy.engine.Engine ROLLBACK


### Ordering or Grouping by a Label

In [68]:
from sqlalchemy import func, desc
stmt = (
    select(Address.user_id, func.count(Address.id).label("num_addresses"))
    .group_by("user_id")
    .order_by("user_id", desc("num_addresses"))
)
print(stmt)

SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_addresses DESC


# Using Aliases